# Benchmarks to try and improve the run time of minlp solver scaling up

In [1]:
%load_ext autoreload
%autoreload 2

In [63]:
import itertools
import numpy as np
from collections import namedtuple
np.random.seed(42)

# For starters, we'll simply benchmark on fixed randomly generated test sets of increasing size, dimension and scale (decreased density).
sizes = [100]
dimensions = [2,3]
scales = [1]

Dataset = namedtuple("Dataset", "data size dimension scale source")

datasets = {(i,j,k): Dataset(np.random.rand(i,j) * k, i, j, k, "random") for i,j,k in itertools.product(sizes, dimensions, scales)}

Note: Below we're setting values for the min_volume and absolute errors that are based on the fact that we here know the underlying distribution to be uniform, which means that the true underlying $f(h)$ value for all patterns $h$ is actually $1$. We really only care about compuational effects, but the hope is that these are roughly the kinds of ranges that are relevant when dealing with actual anomaly detection problems and other distributions

In [64]:
import pyomo
from typing import Callable
import time
from rare_pattern_detect.minlp_based import MINLPModel

Parameter = namedtuple("Parameter", "name value")
# solver parameters
bound_included = [Parameter("bound included", v) for v in [0.05, 1]]
initial_patterns = [Parameter("initial pattern", v) for v in ["minimal", "maximal"]]
min_volumes = [Parameter("min volume", v) for v in [0.05, 0.1]]
absolute_errors = [Parameter("absolute error", v) for v in [1e-3, 0.1]] # thinking this through I believe whatever absolute error we set, we can simply add it to the epsilon of the pac performance.
relative_errors = [Parameter("relative error", v) for v in [1e-3, 0.1]] # this becomes useful especially if we set epsilon to be relative to an estimate of f.
# use_parallel_threads = [2, 4, 6]
# use_different_solvers = [Falase, ]

parameters = [bound_included,
              initial_patterns,
              min_volumes,
              absolute_errors,
              relative_errors
              ]

def generate_solver_settings(in_dict):
    solver_settings = {"tee": False}
    if "absolute error" in in_dict:
        solver_settings["absolute_bound_tolerance"] = in_dict["absolute error"]
    if "relative error" in in_dict:
        solver_settings["relative_bound_tolerance"] = in_dict["relative error"]
    return solver_settings

def run_on_testset(data, testdata, **kwargs):
    solutions = []
    optimal_count = 0
    for point in testdata:
        model = MINLPModel(data, min_volume="kwargs", **kwargs)
        status, result = model.find_min_f_hat(point, solver_settings=generate_solver_settings(kwargs))
        if status == "ok":
            solutions.append(result)
            optimal_count += 1
    return solutions, optimal_count, model.solver_settings

def run_on_whole_dataset(data):
    return run_on_testset(data, data)

def run_on_fixed_size_sample(data, size, **kwargs):
    np.random.seed(0)
    N = len(data)
    assert size <= N, "size larger than dataset"
    sample_indices = np.random.choice(N, max(1, size))
    return run_on_testset(data, data[sample_indices], **kwargs)

def run_on_fraction(data, fraction, **kwargs):
    return run_on_fixed_size_sample(data, round(len(data)*fraction), **kwargs)

def benchmark_and_store_result(expression: Callable):
    start = time.time()
    res = expression.__call__()
    end = time.time()
    return *res, end - start

In [65]:
import mlflow

mlflow.set_experiment("MINLP Tweaking Small after debugging")
run_number = 0
for parameter_combo in itertools.product(*parameters):
    for dataset in datasets.values():
        run_number += 1
        print(f"Running experiment number {run_number} for dataset {(dataset.size, dataset.dimension)} with parameters {[(p.name, p.value) for p in parameter_combo]}")
        with mlflow.start_run():

            # dataset parameters
            mlflow.log_param("size", dataset.size)
            mlflow.log_param("dimension", dataset.dimension)
            mlflow.log_param("density", dataset.scale)
            mlflow.log_param("source", dataset.source)

            # option parameters
            for parameter in parameter_combo:
                mlflow.log_param(parameter.name, parameter.value)

            parameter_dict = {parameter.name : parameter.value for parameter in parameter_combo}

            # min_volume is expressed as fraction of scale
            parameter_dict["min volume"] *= dataset.scale

            # timing results
            sample_size = 10
            f_hats, optimal_count, solver_settings, time_passed = benchmark_and_store_result(lambda: run_on_fixed_size_sample(dataset.data,sample_size, **parameter_dict))
            performance = np.mean(f_hats)
            mlflow.log_metric("time", time_passed)
            mlflow.log_metric("optimal count", optimal_count)
            mlflow.log_metric("average f_hat", performance)
            print(f"Results: average f_hat: {performance}, time: {time_passed}")

            for k,v in solver_settings.items():
                mlflow.log_param(k,v)

---------------------------------------------------------------------------------------------
              Mixed-Integer Nonlinear Decomposition Toolbox in Pyomo (MindtPy)               
---------------------------------------------------------------------------------------------
For more information, please visit https://pyomo.readthedocs.io/en/stable/contributed_packages/mindtpy.html
Original model has 911 constraints (4 nonlinear) and 0 disjunctions, with 506 variables, of which 500 are binary, 0 are integer, and 6 are continuous.
Objective is nonlinear. Moving it to constraint set.
rNLP is the initial strategy being used.

 Iteration | Subproblem Type | Objective Value | Primal Bound |   Dual Bound |   Gap   | Time(s)



Running experiment number 1 for dataset (100, 2) with parameters [('bound included', 0.05), ('initial pattern', 'minimal'), ('min volume', 0.05), ('absolute error', 0.001), ('relative error', 0.001)]


         -       Relaxed NLP           1.00967            inf        1.00967      nan%      0.22
         1              MILP           1.96927            inf        1.96927      nan%      0.33
NLP subproblem was locally infeasible.
Solving feasibility problem
         2              MILP           2.01401            inf        2.01401      nan%      0.94
NLP subproblem was locally infeasible.
Solving feasibility problem
         3              MILP           2.33984            inf        2.33984      nan%      1.45
NLP subproblem was locally infeasible.
Solving feasibility problem
         4              MILP           2.58196            inf        2.58196      nan%      1.95
NLP subproblem was locally infeasible.
Solving feasibility problem
         5              MILP           16.9912            inf        16.9912      nan%      2.58
NLP subproblem was locally infeasible.
Solving feasibility problem
         6              MILP           33.3025            inf        33.3025      n

Results: average f_hat: 30.699590554944013, time: 39.1971800327301
Running experiment number 2 for dataset (100, 3) with parameters [('bound included', 0.05), ('initial pattern', 'minimal'), ('min volume', 0.05), ('absolute error', 0.001), ('relative error', 0.001)]


         -       Relaxed NLP          0.997827            inf       0.997827      nan%      0.28
         1              MILP           1.96562            inf        1.96562      nan%      0.48
NLP subproblem was locally infeasible.
Solving feasibility problem
         2              MILP           1.97665            inf        1.97665      nan%      1.38
NLP subproblem was locally infeasible.
Solving feasibility problem
         3              MILP           1.98977            inf        1.98977      nan%      1.89
NLP subproblem was locally infeasible.
Solving feasibility problem
         4              MILP           2.49946            inf        2.49946      nan%      3.56
NLP subproblem was locally infeasible.
Solving feasibility problem
         5              MILP           2.52071            inf        2.52071      nan%      6.45
NLP subproblem was locally infeasible.
Solving feasibility problem
         6              MILP           3.14789            inf        3.14789      n

...

Results: average f_hat: 15.694976359673134, time: 167.1638307571411
Running experiment number 63 for dataset (100, 2) with parameters [('bound included', 1), ('initial pattern', 'maximal'), ('min volume', 0.1), ('absolute error', 0.1), ('relative error', 0.1)]


         -       Relaxed NLP           1.00967            inf        1.00967      nan%      0.22
         1              MILP           1.96927            inf        1.96927      nan%      0.33
NLP subproblem was locally infeasible.
Solving feasibility problem
         2              MILP           2.01401            inf        2.01401      nan%      0.77
NLP subproblem was locally infeasible.
Solving feasibility problem
         3              MILP           2.58196            inf        2.58196      nan%      1.28
NLP subproblem was locally infeasible.
Solving feasibility problem
         4              MILP           18.3722            inf        18.3722      nan%      2.00
NLP subproblem was locally infeasible.
Solving feasibility problem
         5              MILP           59.3012            inf        59.3012      nan%      2.97
*        5         Fixed NLP           39.0127        39.0127        59.3012   -52.00%      3.15
MindtPy exiting on bound convergence. Absolute gap: -

Results: average f_hat: 48.81170052086171, time: 34.67243194580078
Running experiment number 64 for dataset (100, 3) with parameters [('bound included', 1), ('initial pattern', 'maximal'), ('min volume', 0.1), ('absolute error', 0.1), ('relative error', 0.1)]


         -       Relaxed NLP          0.997827            inf       0.997827      nan%      0.28
         1              MILP           1.96562            inf        1.96562      nan%      0.47
NLP subproblem was locally infeasible.
Solving feasibility problem
         2              MILP           1.97665            inf        1.97665      nan%      0.94
NLP subproblem was locally infeasible.
Solving feasibility problem
         3              MILP           1.98977            inf        1.98977      nan%      1.44
NLP subproblem was locally infeasible.
Solving feasibility problem
         4              MILP           3.59446            inf        3.59446      nan%      2.06
NLP subproblem was locally infeasible.
Solving feasibility problem
         5              MILP           3.71105            inf        3.71105      nan%      2.70
NLP subproblem was locally infeasible.
Solving feasibility problem
         6              MILP           4.59472            inf        4.59472      n

Results: average f_hat: 15.699403115522708, time: 135.5073161125183


In [ ]:
!mlflow ui

# Results for the isolation forest

Here, instead of running the exact solver, we use the sampling procedure from the isolation forest, but we use family of scoring functions that are based on the sampled distribution of f_hat.

In [ ]:
from sklearn.ensemble import IsolationForest

class IFBasedRarePatternDetect(IsolationForest):
    def __init__(self):
        super().__init__()

    def get_pattern_space(self, point) -> :
